In [ ]:
import requests
import pandas as pd

full_df = pd.read_csv('codes_raw.csv')
full_df = full_df[full_df['iata_code'].notna()]

strip_df = full_df.copy()
strip_df = strip_df[strip_df['name'].str.contains('strip')]

strip_codes = strip_df['iata_code'].tolist()

not_found_list = []
for i in range(len(strip_codes)):
    input_code = strip_codes[i]

    url = f'https://www.travelocity.com/api/v4/typeahead/{input_code}?browser=Chrome&client=Homepage&dest=true&device=Desktop&expuserid=-1&features=nearby_airport%7Cta_hierarchy%7Cconsistent_display&format=json&guid=7da5874a-d90e-4acf-9531-b340a1d2e416&lob=FLIGHTS&locale=en_US&maxresults=10&personalize=true&regiontype=4959&siteid=80001&ab=42716.1'
    res = requests.get(url).json()
    output_code = res['sr'][0]['hierarchyInfo']['airport']['airportCode']

    if input_code != output_code:
        not_found_list.append(input_code)

final_df = full_df[~full_df['iata_code'].isin(not_found_list)]

final_df['lng'] = final_df['coordinates'].astype(
    str).str.split(',', expand=True)[0].str.replace(' ', '')

final_df['lat'] = final_df['coordinates'].astype(
    str).str.split(',', expand=True)[1].str.replace(' ', '')


final_strings = []
for i in range(len(final_df)):
    row = final_df.iloc[i]
    code = {
        'code': row['iata_code'],
        'name1': row['name'],
        'name2': f"{row['iata_code']} - {row['name']}",
        'lat': row['lat'], 'lng': row['lng']
    }
    final_strings.append(code)

final_strings.head(10)


In [ ]:
import json
with open('filtered_codes.json', 'w') as f:
    json.dump(final_strings, f)
